In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl

In [6]:
# !pip install torchsummary
# !pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 45.6 MB/s eta 0:00:00


In [7]:
from torchmetrics import Accuracy
from torchsummary import summary

In [8]:
# Hyper parameters
dropout_value = 0.1

In [11]:
class LitCustomResNet(pl.LightningModule):
    def __init__(self):
        super(LitCustomResNet,self).__init__()
        self.acc = Accuracy(task='multiclass',num_classes=10)

        # PREP LAYER: CONVOLUTION BLOCK 1 input 32/1/1
        self.preplayer1 = nn.Sequential(
            nn.Conv2d(3,64,3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(dropout_value)
            ) # output_size = 32/3/1 # of kernels: 64

        # LAYER 1
        self.convblock1 = nn.Sequential(
            nn.Conv2d(64,128,3,padding=1),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(dropout_value)
            ) # output_size =

        self.resblock1 = nn.Sequential(
            nn.Conv2d(128,128,3,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(dropout_value)
            ) # output_size =

        self.resblock2 = nn.Sequential(
            nn.Conv2d(128,128,3,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(dropout_value)
            ) # output_size =
        ## Layer 1 Ends
        # LAYER 2
        self.convblock2 = nn.Sequential(
            nn.Conv2d(128,256,3,padding=1),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(dropout_value)
            ) # output_size =
        ## Layer 2 Ends
        # LAYER 3
        self.convblock3 = nn.Sequential(
            nn.Conv2d(256,512,3,padding=1),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(dropout_value)
            ) # output_size =

        self.resblock3 = nn.Sequential(
            nn.Conv2d(512,512,3,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(dropout_value)
            ) # output_size =

        self.resblock4 = nn.Sequential(
            nn.Conv2d(512,512,3,padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(dropout_value)
            ) # output_size =
        ## Layer 3 Ends
        # maxpool
        self.maxpoollayer1 = nn.Sequential(
            nn.MaxPool2d(4,2)
        ) # output_size = 1/69

        self.ffconnectedlayer1 = nn.Linear(
            in_features=512,
            out_features=10,
            bias=False) # 1*1*10/69/4
        self.dropout = nn.Dropout(dropout_value)

    def forward(self,x):
        #print('input shape is:',x.shape)
        ## pre layer ###
        x = self.preplayer1(x)
        #print('prep layer output is:',x.shape)
        ## layer 1 ###
        x = self.convblock1(x)
        out1 = x
        #print('layer1 block1 output is:',x.shape)
        x = self.resblock1(x)
        #print('layer1 resblock1 output is:',x.shape)
        x = self.resblock2(x)
        #print('layer1 resblock2 output is:',x.shape)
        x = out1 + x
        #print('layer1 final output is:',x.shape)
        ########### layer 2 #########
        x = self.convblock2(x)
        #print('layer2 block2 output is:',x.shape)
        ########### Layer 3 #########
        x = self.convblock3(x)
        #print('layer3 block3 output is:',x.shape)
        out2 = x
        x = self.resblock3(x)
        #print('layer3 resblock3 output is:',x.shape)
        x = self.resblock4(x)
        #print('layer3 resblock4 output is:',x.shape)
        x = out2 + x
        #print('layer3 final output is:',x.shape)
        #####  maxpool
        x = self.maxpoollayer1(x)
        #print('maxpool output is:',x.shape)
        x = x.view(-1, 512)
        ## fully connected layer
        x = self.ffconnectedlayer1(x)
        #print('fflayer output is:',x.shape)


        return F.log_softmax(x, dim=-1)